# Project group 30

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.set_printoptions(precision = 4, suppress = True)

In [2]:
#pip install pycountry_convert
from pycountry_convert import country_name_to_country_alpha2

In [3]:
population = 'population.csv'
df_population= pd.read_csv(population)
medals = 'medals.csv'
df_medals= pd.read_csv(medals)
cpi = 'cpi.csv'
df_cpi= pd.read_csv(cpi)

#df_population

In [4]:
import pycountry
import pycountry_convert as pcc
from fuzzywuzzy import fuzz, process

# Create a list of all unique countries in the dataframe
countries = df_medals['Country'].unique().tolist()
print(countries)

#Create a list of all standard country names
countries_standard = [country.name for country in pycountry.countries]
#print(countries_standard)

for i in range(len(df_medals)):
    temp = df_medals['Country'][i]
    country = 'c'
    try:
        #print(pycountry.countries.lookup(temp))
        country = pycountry.countries.lookup(temp)
        #print(country)
    except LookupError as e:
        print('a')
        #continue
    if country == 'c':
        #print('k')
        try:
            match = process.extractOne(str(temp), countries_standard, scorer=fuzz.token_sort_ratio)
            print(match)
            print('kk')
            if match[1] > 74:
                print(match[0])
                c = pycountry.countries.get(name=match[0])
                country_code=c.alpha_3
                #country_code = pcc.country_alpha3_to_country_name(match[0])
                df_medals.loc[i, 'Country'] = country_code
            else:
                df_medals.loc[i, 'Country'] = None
        except ValueError as e:
            df_medals.loc[i, 'Country'] = None
    else:
        df_medals.loc[i, 'Country'] = country.alpha_3
    
#print(countries)
df_medals['Country']

/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


['United States of America', "People's Republic of China", 'Japan', 'Great Britain', 'ROC', 'Australia', 'Netherlands', 'France', 'Germany', 'Italy', 'Canada', 'Brazil', 'New Zealand', 'Cuba', 'Hungary', 'Republic of Korea', 'Poland', 'Czech Republic', 'Kenya', 'Norway', 'Jamaica', 'Spain', 'Sweden', 'Switzerland', 'Denmark', 'Croatia', 'Islamic Republic of Iran', 'Serbia', 'Belgium', 'Bulgaria', 'Slovenia', 'Uzbekistan', 'Georgia', 'Chinese Taipei', 'Turkey', 'Greece', 'Uganda', 'Ecuador', 'Ireland', 'Israel', 'Qatar', 'Bahamas', 'Kosovo', 'Ukraine', 'Belarus', 'Romania', 'Venezuela', 'India', 'Hong Kong, China', 'Philippines', 'Slovakia', 'South Africa', 'Austria', 'Egypt', 'Indonesia', 'Ethiopia', 'Portugal', 'Tunisia', 'Estonia', 'Fiji', 'Latvia', 'Thailand', 'Bermuda', 'Morocco', 'Puerto Rico', 'Colombia', 'Azerbaijan', 'Dominican Republic', 'Armenia', 'Kyrgyzstan', 'Mongolia', 'Argentina', 'San Marino', 'Jordan', 'Malaysia', 'Nigeria', 'Bahrain', 'Saudi Arabia', 'Lithuania', 'Nor

0      USA
1      CHN
2      JPN
3     None
4     None
      ... 
88     GHA
89     GRD
90     KWT
91     MDA
92     SYR
Name: Country, Length: 93, dtype: object

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df_medals)

   Country  Gold Medal  Silver Medal  Bronze Medal  Total  Rank By Total
0      USA          39            41            33    113              1
1      CHN          38            32            18     88              2
2      JPN          27            14            17     58              5
3     None          22            21            22     65              4
4     None          20            28            23     71              3
5      AUS          17             7            22     46              6
6      NLD          10            12            14     36              9
7      FRA          10            12            11     33             10
8      DEU          10            11            16     37              8
9      ITA          10            10            20     40              7
10     CAN           7             6            11     24             11
11     BRA           7             6             8     21             12
12     NZL           7             6             7 

[('Czechia', 100),
 ('Korea, Republic of', 64),
 ('Dominican Republic', 62),
 ('Central African Republic', 61),
 ('Moldova, Republic of', 59)]

['Aruba', 'Afghanistan', 'Angola', 'Anguilla', 'Åland Islands', 'Albania', 'Andorra', 'United Arab Emirates', 'Argentina', 'Armenia', 'American Samoa', 'Antarctica', 'French Southern Territories', 'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Bonaire, Sint Eustatius and Saba', 'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas', 'Bosnia and Herzegovina', 'Saint Barthélemy', 'Belarus', 'Belize', 'Bermuda', 'Bolivia, Plurinational State of', 'Brazil', 'Barbados', 'Brunei Darussalam', 'Bhutan', 'Bouvet Island', 'Botswana', 'Central African Republic', 'Canada', 'Cocos (Keeling) Islands', 'Switzerland', 'Chile', 'China', "Côte d'Ivoire", 'Cameroon', 'Congo, The Democratic Republic of the', 'Congo', 'Cook Islands', 'Colombia', 'Comoros', 'Cabo Verde', 'Costa Rica', 'Cuba', 'Curaçao', 'Christmas Island', 'Cayman Islands', 'Cyprus', 'Czechia', 'Germany', 'Djibouti', 'Dominica', 'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador', 'Egy